In [1]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import building_blocks
from building_blocks import *

from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim.lr_scheduler import ReduceLROnPlateau

from datasets import EliaSolarDataset
from utils import ConfigSettings

In [2]:
cf = ConfigSettings(config_path='config.ini')

In [3]:
solar_dataset = EliaSolarDataset(
        csv_path=cf.data.file_path,
        datetime_column='DateTime',
        target_column="Corrected Upscaled Measurement [MW]",
        context_length=cf.model.context_length,
        frequency=cf.data.frequency,
        train_test_split_year=cf.data.train_test_split_year,
        train_val_split_year=cf.data.train_val_split_year,)

In [4]:
print(solar_dataset.data.shape)

torch.Size([105769, 13, 120])


In [5]:
df = solar_dataset.get_dataframe(preprocessed=True)

In [6]:
df.head()

,DateTime,Corrected Upscaled Measurement [MW],day_of_year,0,1,2,3,4,5,6,7,8,9,10,11
0,2012-02-01 00:00:00,0.0,32,0.351889,0.999517,0.384225,0.019989,0.000141,1.341028e-07,1.729329e-11,3.018061e-16,1.114240e-11,9.434503e-08,0.000108,0.016766
1,2012-02-01 01:00:00,0.0,32,0.351889,0.999517,0.384225,0.019989,0.000141,1.341028e-07,1.729329e-11,3.018061e-16,1.114240e-11,9.434503e-08,0.000108,0.016766
2,2012-02-01 02:00:00,0.0,32,0.351889,0.999517,0.384225,0.019989,0.000141,1.341028e-07,1.729329e-11,3.018061e-16,1.114240e-11,9.434503e-08,0.000108,0.016766
3,2012-02-01 03:00:00,0.0,32,0.351889,0.999517,0.384225,0.019989,0.000141,1.341028e-07,1.729329e-11,3.018061e-16,1.114240e-11,9.434503e-08,0.000108,0.016766
4,2012-02-01 04:00:00,0.0,32,0.351889,0.999517,0.384225,0.019989,0.000141,1.341028e-07,1.729329e-11,3.018061e-16,1.114240e-11,9.434503e-08,0.000108,0.016766


In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [8]:
solar_dataset.train_test_split_index

78287

In [9]:

indices = list(range(len(solar_dataset)))
train_indices = indices[:solar_dataset.train_val_split_index]
val_indices = indices[solar_dataset.train_val_split_index:solar_dataset.train_test_split_index]
test_indices = indices[solar_dataset.train_test_split_index:]

# Creating data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)
test_sampler = SubsetRandomSampler(test_indices)

train_loader = torch.utils.data.DataLoader(solar_dataset, batch_size=cf.model.batch_size, 
                                        sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(solar_dataset, batch_size=cf.model.batch_size,
                                                sampler=valid_sampler)
test_loader = torch.utils.data.DataLoader(solar_dataset, batch_size=cf.model.batch_size,sampler=test_sampler)

In [10]:
# https://stackoverflow.com/questions/65996797/how-to-refresh-a-python-import-in-a-jupyter-notebook-cell
from importlib import reload
reload(building_blocks)
from building_blocks import *

model_params = TimeSeriesTransformerParams(
    input_dim=cf.model.context_length,
    embed_size=cf.model.embedding_size,
    num_layers=cf.model.num_layers,
    heads=cf.model.num_attention_heads,
    device=device,
    forward_expansion=cf.model.forward_expansion,
    dropout=cf.model.dropout,
    forecast_size=cf.model.forecast_size
)
scenario_params = ScenarioParams(
    name="electricity",
    device=device,
    epochs=100,
    dataloader_train=train_loader,
    dataloader_validation=validation_loader,
    dataloader_test=test_loader,
    base_path="/dne" if cf.runtime.run_in_colab else "./"
)
model = TimeSeriesTransformer.from_params(model_params).to(device)
scenario = Scenario(scenario_params)
result = scenario.execute(model)
print(f"execution done")

c:\Repositories\OpenUniversiteit\dne\code\venv\lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


checking paths, base path is ./
batch: {'data': tensor([[[3.0444e-01, 2.8540e-01, 2.8857e-01,  ..., 1.4069e-02,
          4.4800e-02, 9.9915e-02],
         [7.1526e-07, 7.1526e-07, 7.1526e-07,  ..., 2.4438e-06,
          2.4438e-06, 2.4438e-06],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         ...,
         [5.6250e-01, 5.6250e-01, 5.6250e-01,  ..., 7.0312e-01,
          7.0312e-01, 7.0312e-01],
         [4.5441e-02, 4.5441e-02, 4.5441e-02,  ..., 7.8979e-02,
          7.8979e-02, 7.8979e-02],
         [4.9639e-04, 4.9639e-04, 4.9639e-04,  ..., 1.1997e-03,
          1.1997e-03, 1.1997e-03]],

        [[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [1.1921e-06, 1.1921e-06, 1.1921e-06,  ..., 3.9339e-06,
          3.9339e-06, 3.9339e-06],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         ...,
         [6.1914e-01, 6.1914e-0

AttributeError: 'str' object has no attribute 'to'